In [1]:
from ctypes import *
import time
from sys import path
from os import sep
import numpy as np

dwf = cdll.LoadLibrary("libdwf.so")
constants_path = "/usr/share/digilent/waveforms/samples/py"
path.append(constants_path)
import dwfconstants as constants

In [3]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.01V


In [4]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [5]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ENABLE_REG, 0x03) #Now, also RGBC enable / PON = 1, AEN = 1
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16

True

In [6]:
#Function to test the sensor output
def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [7]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [8]:
cleanup()
#call it everytime after work is done

In [38]:
write_register(CONTROL_REG,0)
for i in range(256):
    write_register(ATIME_REG,255-i)
    colors = read_colors()

    print(f"Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Clear: 0, R: 0, G: 0, B: 0
Clear: 0, R: 0, G: 0, B: 0
Clear: 0, R: 0, G: 0, B: 0
Clear: 0, R: 0, G: 0, B: 0
Clear: 0, R: 0, G: 0, B: 0
Clear: 0, R: 0, G: 0, B: 0
Clear: 1, R: 0, G: 0, B: 0
Clear: 1, R: 0, G: 0, B: 0
Clear: 1, R: 0, G: 0, B: 0
Clear: 1, R: 0, G: 0, B: 0
Clear: 1, R: 0, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 1, R: 1, G: 0, B: 0
Clear: 2, R: 1, G: 0, B: 0
Clear: 2, R: 1, G: 0, B: 0
Clear: 2, R: 1, G: 0, B: 0
Clear: 2, R: 1, G: 0, B: 0
Clear: 2, R: 1, G: 0, B: 0
Clear: 2, R: 1, G: 0, B: 0
Clear: 2, R: 1, G: 1, B: 0
Clear: 2, R: 1, G: 1, B: 0
Clear: 2, R: 1, G: 1, B: 0
Clear: 2, R: 1, G: 1, B: 0
Clear: 2, R: 1, G: 1, B: 0
Clear: 2, R: 1, G: 1, B: 0
Clear: 2, R: 1, G: 1, B: 0
Clear: 3, R: 2, G: 1, B: 0
Clear: 3, R: 2, G: 1, B: 0
Clear: 3, R: 2, G: 1, B: 1
Clear: 3, R: 2, G: 1, B: 1
Clear: 3, R: 2, G: 1, B: 1
Clear: 3, R: 2, G: 1, B: 1
C

In [30]:
print("Settings applied. Stabilizing for 1 second...")
time.sleep(1.0) 
print("Stabilizing done, now testing...")
print(" ")

clear_channel_samples = []
for i in range(1000):
    colors = read_colors()
    
    if colors:
        # Save the 'Clear' channel for statistical analysis
        clear_channel_samples.append(colors['clear'])
        
        # Print every 100th sample to avoid flooding the output window
        if i % 100 == 0:
            print(f"Sample {i}: Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

#ANALYSIS
if len(clear_channel_samples) > 0:
    mean_val = np.mean(clear_channel_samples)
    std_dev = np.std(clear_channel_samples)
    noise_percent = (std_dev / mean_val) * 100
    
    print("\n" + "="*40)
    print(f"ANALYSIS RESULT (N=1000)")
    print(f"Mean Count:       {mean_val:.2f}")
    print(f"Standard Dev:     {std_dev:.2f}")
    print(f"Noise (Jitter):   {noise_percent:.3f}%")
    print("="*40)
else:
    print("Error: No data collected.")

Settings applied. Stabilizing for 1 second...
Stabilizing done, now testing...
 
Sample 0: Clear: 21, R: 13, G: 6, B: 4
Sample 100: Clear: 21, R: 13, G: 6, B: 4
Sample 200: Clear: 21, R: 13, G: 6, B: 4
Sample 300: Clear: 21, R: 13, G: 6, B: 4
Sample 400: Clear: 21, R: 13, G: 6, B: 4
Sample 500: Clear: 21, R: 13, G: 6, B: 4
Sample 600: Clear: 21, R: 13, G: 6, B: 4
Sample 700: Clear: 21, R: 13, G: 6, B: 4
Sample 800: Clear: 21, R: 13, G: 6, B: 4
Sample 900: Clear: 21, R: 13, G: 6, B: 4

ANALYSIS RESULT (N=1000)
Mean Count:       21.00
Standard Dev:     0.00
Noise (Jitter):   0.000%


In [40]:
print(colors)
print(clear_channel_samples)

{'clear': 21, 'red': 13, 'green': 6, 'blue': 4}
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,